<a href="https://colab.research.google.com/github/s3xmaxman/whisper-translate/blob/main/faster_whisper_srt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q faster-whisper ipython-autotime
%load_ext autotime

In [ ]:
audio = "test.mp3"

In [ ]:
# srt file を作成するための関数
def create_srt_file(results, use_faster_whisper, file_name="transcribe"):
    data = []
    with open(f"{file_name}.srt", mode="w") as f:
        for index, _dict in enumerate(results):
            if use_faster_whisper:
              start_time = _dict.start # start
              end_time = _dict.end # end
              text = _dict.text # text
            else:
              start_time = _dict["start"]
              end_time = _dict["end"]
              text = _dict["text"]

            data.append({
            "index": index + 1,
            "start": start_time,
            "end": end_time,
            "text": text})

            # 時、分、秒、ミリ秒に分割
            s_h, s_m, s_s = int(start_time // 3600), int((start_time % 3600) // 60), int(start_time % 60)
            e_h, e_m, e_s = int(end_time // 3600), int((end_time % 3600) // 60), int(end_time % 60)

            # ミリ秒を計算
            s_ms = int((start_time - int(start_time)) * 1000)
            e_ms = int((end_time - int(end_time)) * 1000)

            f.write(f"{index+1}\n")
            f.write(f"{s_h:02}:{s_m:02}:{s_s:02},{s_ms:03} --> {e_h:02}:{e_m:02}:{e_s:02},{e_ms:03}\n")
            f.write(f"{text}\n\n")
            print(f"{index+1}")
            print(f"{s_h:02}:{s_m:02}:{s_s:02},{s_ms:03} --> {e_h:02}:{e_m:02}:{e_s:02},{e_ms:03}")
            print(f"{text}")
            print()
    return data

In [ ]:
from faster_whisper import WhisperModel

model_size = "large-v3"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

In [ ]:
segments, info = model.transcribe(audio, beam_size=5)

In [ ]:
faster_whisper_data = create_srt_file(results=segments, use_faster_whisper=True, file_name="output")